### Подготовка датасета по показателю объем убоя домашней птицы

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

from pylab import rcParams
from IPython.display import display
import math
from prophet import Prophet
pd.set_option('display.max_columns', 130)


import warnings
from statsmodels.tools.sm_exceptions import InterpolationWarning
warnings.simplefilter("ignore", category=InterpolationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)



In [2]:
df = pd.read_excel("../../Data cleansing/output data/Просуммированные по категориям с доп регрессорами.xlsx")
df.head(5)

,Показатель,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,Верблюды,АКМОЛИНСКАЯ ОБЛАСТЬ,0.00,0.00,0.40,0.00,0.00,0.00,0.09,1.00,0.00,0.00,0.20,0.00,0.00,0.18,0.28,0.00,0.00,0.40,0.00,0.00,0.65,0.00,0.31,1.00,0.00,0.00,0.00,0.00,0.00,2.01,0.00,1.20,0.00,0.00,2.18,0.39,0.00,0.00,1.04,0.00,0.14,2.08,0.00,0.00,0.00,0.00,0.00,0.30,0.00,0.00,0.00,0.66,0.00,0.33,0.00,0.9,0.00,0.00,0.00,10.08,0.00,0.0,0.00,0.0,0.00,0.54,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.36,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.40,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.00
1,Верблюды,АКТЮБИНСКАЯ ОБЛАСТЬ,101.98,67.47,374.84,115.59,218.72,14.15,19.77,3.00,39.16,46.16,238.56,463.35,109.04,72.94,384.96,114.35,221.89,10.93,18.79,3.50,38.53,46.64,216.08,472.07,110.32,68.17,380.15,127.12,217.61,14.98,21.26,7.14,44.63,51.78,239.43,513.33,115.70,69.40,371.53,130.17,218.41,14.59,21.76,7.19,45.97,55.48,252.68,527.66,117.94,70.37,385.75,117.80,218.45,15.54,21.78,7.2,47.06,58.02,257.54,543.35,119.80,71.3,396.90,121.8,228.10,15.90,21.9,7.2,63.00,59.90,260.20,552.10,121.60,71.8,398.40,128.50,228.80,13.10,22.50,7.20,63.90,61.90,150.70,554.90,121.30,73.3,404.20,128.70,234.90,13.60,23.60,6.3,49.80,62.4,153.70,552.8,118.07,69.17,384.21,121.01,223.46,11.76,21.39,5.52,48.18,61.39,150.87,531.79,119.50,72.60,391.0,126.1,229.10,12.2,24.10,5.7,49.87,63.00,154.3,538.40
2,Верблюды,АЛМАТИНСКАЯ ОБЛАСТЬ,1.00,0.20,51.60,25.40,0.00,61.90,76.87,95.57,16.09,0.10,10.80,46.78,15.06,13.70,131.80,9.20,17.89,130.12,2.00,0.00,4.99,2.06,4.90,44.48,15.06,23.94,41.60,19.54,3.00,15.14,18.50,14.82,14.52,12.60,9.92,43.11,20.72,2.96,15.24,0.00,1.00,20.03,3.44,7.77,117.25,1.00,0.00,20.34,21.95,3.04,17.91,26.81,9.16,18.80,13.37,0.0,17.29,15.90,4.69,23.10,10.85,9.7,42.75,0.0,6.21,21.10,4.0,2.6,28.10,4.53,79.50,93.18,23.70,11.2,16.00,2.00,3.10,19.62,2.30,4.85,11.18,2.80,9.65,36.44,10.60,11.5,26.10,7.20,12.75,11.20,11.20,27.0,25.26,17.9,21.90,12.5,16.55,16.81,22.78,12.10,12.44,6.85,39.65,17.22,8.85,27.48,0.48,29.72,18.90,17.40,12.5,16.4,11.60,16.7,15.70,24.0,7.70,6.90,5.4,12.60
3,Верблюды,АТЫРАУСКАЯ ОБЛАСТЬ,213.89,167.70,306.60,164.97,342.57,192.00,43.60,113.03,262.37,193.97,308.17,1087.33,325.10,190.60,301.10,154.84,328.50,220.30,66.10,118.20,234.90,196.51,270.92,974.18,303.30,167.41,323.54,182.34,349.10,249.40,57.57,88.40,221.21,199.63,278.80,964.02,305.22,159.62,326.87,159.50,367.20,257.50,60.01,78.10,253.27,332.00,355.05,989.86,280.08,165.05,334.51,134.85,367.97,321.22,126.22,93.7,263.10,366.66,348.54,1041.38,293.40,154.8,339.44,143.6,405.80,308.35,88.5,130.5,575.30,458.70,357.62,998.10,292.76,188.3,487.02,129.28,421.44,351.20,95.77,97.54,624.80,631.65,499.30,1179.66,286.67,189.8,513.16,171.46,402.10,432.22,121.53,126.2,637.50,651.7,482.94,1412.2,307.85,200.59,466.60,182.63,414.56,405.19,184.80,154.96,717.92,719.01,522.83,1164.77,323.81,423.76,441.9,219.1,487.69,446.9,191.43,133.5,730.34,709.68,537.6,930.28
4,Верблюды,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,0.30,0.00,0.00,1.14,6.12,4.70

In [3]:
df['Показатель'].unique()

array(['Верблюды', 'КРС', 'Лошади', 'Молоко', 'Овцы и козы', 'Птица',
       'Свиньи', 'Яйца', 'Температура', 'Поголовье: КРС',
       'Поголовье: лошади', 'Поголовье: овцы и козы', 'Поголовье: свиньи',
       'Поголовье: птица домашняя', 'Поголовье: верблюды', 'Осадки',
       'Цена: Говядина', 'Цена: Баранина', 'Цена: Молоко', 'Цена: Яйца'],
      dtype=object)

In [4]:
df_bird = df[df['Показатель'].isin(['Птица', 'Температура', 'Осадки', 'Поголовье: птица домашняя', 'Яйца', 'Цена: Яйца'])]
df_bird.sample(10)

,Показатель,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
167,Яйца,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.873120e+04,1.792860e+04,1.752900e+04,1.605300e+04,1.537600e+04,1.291550e+04,1.311690e+04,1.151490e+04,1.184180e+04,1.700520e+04,1.767910e+04,1.943540e+04,1.952050e+04,1.782720e+04,1.644460e+04,1.402590e+04,1.442820e+04,1.416670e+04,1.449340e+04,1.275530e+04,1.300850e+04,1.824260e+04,2.119070e+04,2.328810e+04,2.201630e+04,2.036550e+04,1.899820e+04,1.753110e+04,1.779810e+04,1.579790e+04,1.541570e+04,1.481760e+04,1.563330e+04,2.090980e+04,2.167990e+04,2.294160e+04,2.175060e+04,2.043760e+04,2.012380e+04,1.791260e+04,1.744020e+04,1.599500e+04,1.720740e+04,1.653740e+04,1.853030e+04,2.350400e+04,2.457140e+04,2.554840e+04,2.287780e+04,2.122300e+04,2.218760e+04,1.970680e+04,1.915570e+04,1.910630e+04,2.050120e+04,1.700760e+04,1.705050e+04,2.190920e+04,2.252810e+04,2.328140e+04,2.075530e+04,1.987220e+04,2.394460e+04,2.456730e+04,2.077110e+04,1.640850e+04,1.891090e+04,1.664240e+04,1.254310e+04,2.136020e+04,2.446970e+04,2.375420e+04,2.234970e+04,2.243740e+04,2.380800e+04,2.284710e+04,2.163990e+04,1.965940e+04,1.957680e+04
406,Цена: Яйца,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+02,1.000000e+02,1.584000e+02,1.584000e+02,1.710720e+02,2.088789e+02,2.088789e+02,2.088789e+02,2.306023e+02,2.306023e+02,2.306023e+02,2.306023e+02,2.306023e+02,2.306023e+02,2.306023e+02,2.306023e+02,2.638091e+02,2.532567e+02,2.565490e+02,2.688634e+02,2.780047e+02,2.746687e+02,2.840074e+02,2.746352e+02,2.784801e+02,2.648345e+02,2.621862e+02,2.682165e+02,2.521235e+02,2.740582e+02,2.948867e+02,3.223111e+02,3.509968e+02,3.731096e+02,3.820642e+02,3.820642e+02,3.763333e+02,3.721936e+02,3.662385e+02,3.629424e+02,4.072213e+02,4.927378e+02,4.991434e+02,4.991434e+02,4.971468e+02,5.026154e+02,4.810030e+02,4.588768e+02,3.973873e+02,3.973873e+02,4.053351e+02,4.628927e+02,4.971467e+02,5.284670e+02,5.427356e+02,5.579322e+02,5.478894e+02,5.407668e+02,5.396853e+02,5.391456e+02,4.286208e+02,4.341928e+02,4.337586e+02,4.337586e+02,4.701944e+02,4.894723e+02,6.162457e+02,6.710915e+02,6.898821e+02,6.381409e+02,5.979381e+02,4.089896e+02,4.830168e+02,4.830168e+02,4.830168e+02,4.892960e+02,5.303968e+02,5.468391e+02,5.993357e+02
284,Поголовье: птица домашняя,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,3420122.0,3.610458e+06,3.643041e+06,3.735974e+06,3.509041e+06,4.023589e+06,4.177986e+06,4.121550e+06,3.945097e+06,3.901846e+06,3.530169e+06,3.758247e+06,3.728982e+06,3.653991e+06,3.633256e+06,3.906895e+06,3.946815e+06,4.212410e+06,4.430436e+06,4.237303e+06,4.225470e+06,4.453161e+06,4.139825e+06,3.795116e+06,3.746158e+06,4.119676e+06,3.875638e+06,4.126267e+06,4.087333e+06,4.607467e+06,4.414024e+06,4.626857e+06,4.420183e+06,4.313472e+06,4.145387e+06,3.887707e+06,4.047979e+06,3.945586e+06,3.965303e+06,3.9941

In [6]:
# Step 1: Pivot to wide format (each indicator becomes columns of periods)
df_wide = df_bird.pivot(index="Регион", columns="Показатель")

# Step 2: Flatten multi-level columns: ('2015-01', 'КРС') → 'КРС_2015-01'
df_wide.columns = [f"{col[1]}_{col[0]}" for col in df_wide.columns]
df_wide = df_wide.reset_index()

# Step 3: Melt: one row per region-period-indicator
df_melted = df_wide.melt(id_vars="Регион", var_name="indicator_period", value_name="value")

# Step 4: Extract 'Период' and 'Показатель' from the combined column
df_melted["Период"] = df_melted["indicator_period"].str.extract(r"_(\d{4}-\d{2})$")
df_melted["Показатель"] = df_melted["indicator_period"].str.extract(r"^(.+)_\d{4}-\d{2}")

# Step 5: Pivot again to get final modeling format: one row per region+period, one column per indicator
df_bird = df_melted.pivot_table(index=["Регион", "Период"], columns="Показатель", values="value").reset_index()
print(df_bird.groupby("Регион").size().reset_index(name="Количество строк"))
df_bird

                            Регион  Количество строк
0              АКМОЛИНСКАЯ ОБЛАСТЬ               120
1              АКТЮБИНСКАЯ ОБЛАСТЬ               120
2              АЛМАТИНСКАЯ ОБЛАСТЬ               120
3               АТЫРАУСКАЯ ОБЛАСТЬ               120
4   ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ               120
5                          ГАЛМАТЫ               120
6                          ГАСТАНА               120
7                         ГШЫМКЕНТ                79
8               ЖАМБЫЛСКАЯ ОБЛАСТЬ               120
9    ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ               120
10          КАРАГАНДИНСКАЯ ОБЛАСТЬ               120
11            КОСТАНАЙСКАЯ ОБЛАСТЬ               120
12          КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ               120
13           МАНГИСТАУСКАЯ ОБЛАСТЬ               120
14                    ОБЛАСТЬ АБАЙ                31
15                  ОБЛАСТЬ ЖЕТІСУ                31
16                  ОБЛАСТЬ ҰЛЫТАУ                31
17            ПАВЛОДАРСКАЯ ОБЛАСТЬ            

Показатель,Регион,Период,Осадки,Поголовье: птица домашняя,Птица,Температура,Цена: Яйца,Яйца
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-01,9.8,4436655.0,672.81,-12.490323,100.600000,51667.7
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-02,9.8,4446764.0,658.57,-10.192857,88.829800,49357.2
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-03,8.3,4461524.0,469.77,-5.870968,87.053204,59105.8
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-04,8.8,4575245.0,813.94,4.490000,87.227310,61953.7
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-05,42.8,4778690.0,534.59,14.574194,69.607394,72700.2
...,...,...,...,...,...,...,...,...
2166,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,0.0,2167498.0,700.60,27.874194,483.016761,23808.0
2167,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,0.5,2092996.0,711.12,20.766667,489.295979,22847.1
2168,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-10,13.6,2088086.0,691.74,13.200000,530.396841,21639.9
2169,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-11,12.1,2030035.0,667.30,6.500000,546.839143,19659.4


In [7]:
df_bird = df_bird[df_bird["Регион"] != 'РЕСПУБЛИКА КАЗАХСТАН']
df_bird = df_bird.drop(columns=['Осадки', 'Поголовье: птица домашняя', 'Температура', 'Цена: Яйца', 'Яйца'])
df_bird.sample(10)

Показатель,Регион,Период,Птица
573,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,2022-10,6548.09
678,ГАЛМАТЫ,2021-07,0.00
778,ГАСТАНА,2019-11,0.00
178,АКТЮБИНСКАЯ ОБЛАСТЬ,2019-11,143.60
1636,МАНГИСТАУСКАЯ ОБЛАСТЬ,2024-10,601.52
1010,ЖАМБЫЛСКАЯ ОБЛАСТЬ,2022-08,1067.83
2029,СЕВЕРО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,2019-10,816.09
111,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,10894.97
1800,ПАВЛОДАРСКАЯ ОБЛАСТЬ,2020-09,481.53
863,ГШЫМКЕНТ,2020-05,246.40


In [8]:
df_bird.to_excel("Датасет по птице.xlsx", index=False)